In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns

# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.gen_tables.gen_pair_configs as gen_pair

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 

In [3]:
def draw_table(df, cols, hide_index=True):
    if hide_index:
        return df[cols].style.set_table_styles(fmt.table_style).hide_index()
    else:
        return df[cols].style.set_table_styles(fmt.table_style)

# Seq

In [52]:
# run python scripts to generate all the pickles needed
gen_seq = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_seq.py')   
%run $gen_seq 

df_seq = pd.read_pickle(os.path.join(PKL_PATH, 'seq.pkl'))
print(df_seq.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'empty_warp', 'stall_warp', 'idle_warp',
       'scoreboard_warp', 'tot_warp_insn', 'regs', 'smem', 'mem_count',
       'dram_eff', 'dram_bw', 'row_buffer_locality', 'mrqq', 'total_cmd',
       'wasted_col', 'wasted_row', 'mem_idle', 'CCDLc', 'WTRc', 'RTWc', 'RCDc',
       'RCDWRc', 'avg_dram_bw', 'avg_dram_eff', 'avg

In [54]:
col_seq = ['pair_str', 'config', 'runtime', 'instructions', 'ipc', 'avg_dram_bw', 
           'avg_dram_eff', 'avg_mem_lat', 'MPKI', 'waves',
           'avg_row_locality',
#            'l2_total_accesses', 'l2_access_density',
          ]
df_seq.sort_values('pair_str', inplace=True)
df_seq.reset_index(inplace=True, drop=True)
draw_table(df_seq, col_seq, False).format(
    {'avg_dram_bw':'{:.4f}', 
     'ratio_dram_bw': '{:.2f}',
     'MPKI': '{:.2f}',
     'waves': '{:.2f}',
     'l2_total_accesses': '{:,}',
     'l2_access_density': '{:.1f}',
     'instructions': '{:,}',
     'runtime': '{:,}',
    })

,pair_str,config,runtime,instructions,ipc,avg_dram_bw,avg_dram_eff,avg_mem_lat,MPKI,waves,avg_row_locality
0,cut_sgemm-0,TITANV-SEP_RW-PAE,"93,663","228,392,960",2438.4,0.0889,0.470825,302,1.24,0.80,0.459663
1,cut_sgemm-1,TITANV-SEP_RW-PAE,"1,716,792","6,580,076,544",3832.77,0.0747,0.440921,931,0.34,3.20,0.35433
2,cut_wmma-0,TITANV-SEP_RW-PAE,"52,212","36,372,480",696.604,0.0194,0.135825,354,2.27,0.40,0.748845
3,cut_wmma-1,TITANV-SEP_RW-PAE,"1,644,770","2,045,247,488",1243.48,0.4444,0.462079,1116,6.09,3.20,0.178056
4,nvd_binomial-0,TITANV-SEP_RW-PAE,"1,582,948","6,133,492,736",3874.72,0.0000,0.168171,204,0.00,0.80,0.692014
5,nvd_blackscholes-0,TITANV-SEP_RW-PAE,"185,145","292,000,000",1577.13,0.7682,0.824842,3312,8.56,12.21,0.114889
6,nvd_fdtd3d-0,TITANV-SEP_RW-PAE,"2,220,291","3,172,739,184",1428.97,0.4309,0.451833,317,5.57,3.60,0.143883
7,nvd_interval-0,TITANV-SEP_RW-PAE,"58,271","97,138,283",1666.95,0.0224,0.255238,274,2.12,1.07,0.605311
8,nvd_sobol-0,TITANV-SEP_RW-PAE,"156,079","690,560,000",4424.37,0.4157,0.621692,270,1.81,20.00,0.360835
9,parb_cutcp-0,TITANV-SEP_RW-PAE,"524,488","469,895,962",895.91,0.0000,0.100289,154,0.02,0.09,0.782407


# Intra

In [55]:
gen_intra = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_intra.py')
%run $gen_intra

In [56]:
df_intra = pd.read_pickle(os.path.join(PKL_PATH, 'intra.pkl'))
print(df_intra.shape)
col_intra = ['pair_str', 'intra', 'l2', 'norm_ipc']

# draw_table(df_intra, col_intra)

(604, 74)


In [57]:
df_intra_best = pd.read_pickle(os.path.join(PKL_PATH, 'intra_best.pkl'))
df_intra_best.sort_values('pair_str', inplace=True)
col_intra_best = ['pair_str', 
        'perfdollar', 'intra', 'l2', 'norm_ipc', 
        'runtime',
        #'l2_miss_rate',
        #'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
        'comp_busy', 'dram_busy', 'avg_dram_bw', 'avg_dram_eff']

draw_table(df_intra_best, col_intra_best).format({'norm_ipc': "{:.4f}", 'dominant_resc': '{:.2f}'})

pair_str,perfdollar,intra,l2,norm_ipc,runtime,comp_busy,dram_busy,avg_dram_bw,avg_dram_eff
cut_sgemm-0,0.643053,2,0.5,0.9302,100687,0.252251,0.216303,0.124862,0.502992
cut_sgemm-1,1.4245,1,0.5,1.1135,1541784,0.460146,0.220016,0.113804,0.405496
cut_wmma-0,1.5934,2,0.5,0.9440,55309,0.0677628,0.196103,0.0524496,0.171571
cut_wmma-1,0.60502,2,0.5,0.8820,1864897,0.119747,0.945516,0.406337,0.417758
nvd_binomial-0,1.55112,4,0.5,0.8535,1854611,0.398993,9.81654e-05,2.68975e-05,0.129908
nvd_blackscholes-0,0.75747,4,0.5,0.9934,186379,0.155211,0.958107,0.777104,0.809475
nvd_fdtd3d-0,0.604441,1,0.5,0.9822,2260455,0.178864,0.942022,0.445637,0.467388
nvd_interval-0,2.36972,4,0.5,1.0003,58256,0.186245,0.0922248,0.0333817,0.270438
nvd_sobol-0,0.826607,8,0.5,0.9386,166294,0.425753,0.739454,0.416108,0.499896
parb_cutcp-0,3.2715,2,0.5,1.0000,524488,0.127647,8.49003e-05,2.22065e-05,0.100289


# Algorithm: Find pair configs

In [61]:
col_prod = ['norm_ipc_x', 'norm_ipc_y', 'diff_mflat', 'sum_ipc', 
            'intra_x', 'intra_y', 'l2_x', 'l2_y',
           'sum_comp', 'sum_dram', 'penalized']
find_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_pair_configs.py')


app1 = widgets.Dropdown(
    options=df_seq['pair_str'],
    value='cut_sgemm-0',
    rows=5,
    description='App 1:',
    disabled=False
)

app2 = widgets.Dropdown(
    options=df_seq['pair_str'],
    value='cut_sgemm-0',
    rows=5,
    description='App 2:',
    disabled=False
)

qos = widgets.FloatSlider(
    value=0.75,
    min=0.1,
    max=0.95,
    step=0.05,
    description='QoS:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

button = widgets.Button(description='Calculate', button_style='info')

def show_widgets():
    display(app1)
    display(app2)
    display(qos)
    display(button)
    
show_widgets()

def onclick(b):
    clear_output()
    show_widgets()
    %run $find_pair --app $app1.value $app2.value --qos $qos.value --top
    
    df_prod = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
    print(df_prod.columns)
#     sns.lineplot('diff_mflat', 'sum_ipc', data=df_prod)
   
    display(HTML(draw_table(df_prod, col_prod, False).render()))
    
    
button.on_click(onclick)

Dropdown(description='App 1:', options=('cut_sgemm-0', 'cut_sgemm-1', 'cut_wmma-0', 'cut_wmma-1', 'nvd_binomia…

Dropdown(description='App 2:', index=5, options=('cut_sgemm-0', 'cut_sgemm-1', 'cut_wmma-0', 'cut_wmma-1', 'nv…

FloatSlider(value=0.75, continuous_update=False, description='QoS:', max=0.95, min=0.1, step=0.05)

Button(button_style='info', description='Calculate', style=ButtonStyle())

Index(['level_0', 'index', 'pair_str_x', 'config_x', 'gpusim_version_x',
       'jobId_x', 'grid_x_x', 'grid_y_x', 'grid_z_x', 'block_x_x',
       ...
       'thread_ratio_y', 'smem_ratio_y', 'reg_ratio_y', 'usage_y',
       'diff_mflat', 'sum_ipc', 'sum_comp', 'sum_dram', 'penalized', 'config'],
      dtype='object', length=157)


,norm_ipc_x,norm_ipc_y,diff_mflat,sum_ipc,intra_x,intra_y,l2_x,l2_y,sum_comp,sum_dram,penalized
0,0.783434,0.94846,68,1.73189,1,4,0.75,0.25,0.36523,1.14348,nvd_blackscholes-0
1,0.776742,1.00729,76,1.78403,1,5,0.5,0.5,0.372574,1.16608,cut_sgemm-0
2,0.773688,1.02293,88,1.79662,1,4,0.25,0.75,0.373303,1.18643,nvd_blackscholes-0
3,0.773688,1.02966,96,1.80335,1,5,0.25,0.75,0.374822,1.18479,cut_sgemm-0
4,0.773688,1.03565,299,1.80934,1,6,0.25,0.75,0.37541,1.18685,cut_sgemm-0
5,0.773688,1.02988,552,1.80356,1,7,0.25,0.75,0.37466,1.18227,cut_sgemm-0


In [59]:
list_pairs = []
list_app_pairs = []
qos = 0.75

for bench1 in const.app_for_pair:
    for bench2 in const.app_for_pair:
        if bench1 < bench2:
            list_app_pairs.append([bench1, bench2])
#             config_str = gen_pair.main(['--app', bench1, bench2])
#             if len(config_str) > 0: 
#                 df_first = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
#                 list_pairs.append(df_first.head(1).reset_index(drop=True))
            

# df_pairs = pd.concat(list_pairs, axis=0).reset_index()
# df_pairs.sort_values(['sum_dram', 'sum_comp'], ascending=False, inplace=True)
# draw_table(df_pairs, ['pair_str_x', 'pair_str_y'] + col_prod, False)

for idx, p in enumerate(list_app_pairs):
      print(idx, p)

0 ['cut_sgemm-1', 'cut_wmma-0']
1 ['cut_sgemm-1', 'parb_cutcp-0']
2 ['cut_sgemm-1', 'parb_stencil-0']
3 ['cut_sgemm-1', 'parb_lbm-0']
4 ['cut_sgemm-1', 'parb_spmv-0']
5 ['cut_sgemm-1', 'rod_heartwall-0']
6 ['cut_sgemm-1', 'rod_hotspot-0']
7 ['cut_sgemm-1', 'rod_hotspot3d-0']
8 ['cut_sgemm-1', 'rod_streamcluster-0']
9 ['cut_sgemm-1', 'rod_pathfinder-0']
10 ['cut_sgemm-1', 'rod_lavamd-0']
11 ['cut_sgemm-1', 'nvd_binomial-0']
12 ['cut_sgemm-1', 'nvd_blackscholes-0']
13 ['cut_sgemm-1', 'nvd_fdtd3d-0']
14 ['cut_sgemm-1', 'nvd_interval-0']
15 ['cut_sgemm-1', 'nvd_sobol-0']
16 ['cut_wmma-0', 'parb_cutcp-0']
17 ['cut_wmma-0', 'parb_stencil-0']
18 ['cut_wmma-0', 'parb_lbm-0']
19 ['cut_wmma-0', 'parb_spmv-0']
20 ['cut_wmma-0', 'rod_heartwall-0']
21 ['cut_wmma-0', 'rod_hotspot-0']
22 ['cut_wmma-0', 'rod_hotspot3d-0']
23 ['cut_wmma-0', 'rod_streamcluster-0']
24 ['cut_wmma-0', 'rod_pathfinder-0']
25 ['cut_wmma-0', 'rod_lavamd-0']
26 ['cut_wmma-0', 'nvd_binomial-0']
27 ['cut_wmma-0', 'nvd_blackschol